In [28]:
import pandas as pd
import plotly.express as px
import aqi as aqi

In [29]:
# read in data
data_file = "waqi-covid19-airqualitydata-2020.csv"
df_2020 = pd.read_csv(data_file)
df_2020['Date'] = df_2020['Date'].astype('datetime64[ns]')

df_2019Q4 = pd.read_csv("waqi-covid19-airqualitydata-2019Q4.csv")
# df_2019Q4['Date'] = df_2019Q4['Date'].astype('datetime64[ns]')


In [32]:
df_2020

,Unnamed: 0,Date,Country,City,Specie,count,min,max,median,variance
0,0,2020-01-03,IE,Dublin,o3,26,24.5,57.7,49.2,810.59
1,1,2020-01-09,IE,Dublin,o3,19,2.4,31.1,13.4,606.87
2,2,2020-01-10,IE,Dublin,o3,24,4.8,30.2,22.4,479.78
3,3,2020-02-10,IE,Dublin,o3,42,23.2,33.5,29.8,69.84
4,4,2020-02-11,IE,Dublin,o3,39,18.8,32.8,29.7,99.06
...,...,...,...,...,...,...,...,...,...,...
624565,624565,2020-01-15,HU,Budapest,humidity,360,90.0,100.0,91.9,10.90
624566,624566,2020-01-22,HU,Budapest,humidity,360,79.7,100.0,86.0,101.12
624567,624567,2020-01-12,HU,Budapest,humidity,360,65.0,96.0,87.4,335.54
624568,624568,2020-01-18,HU,Budapest,humidity,360,87.5,100.0,91.0,47.53


In [33]:
df_2019Q4

,Date,Country,City,Specie,count,min,max,median,variance
0,11/2/2019,HU,Debrecen,o3,72,1.9,12.2,7.0,59.60
1,11/11/2019,HU,Debrecen,o3,66,0.6,15.2,7.1,151.12
2,11/12/2019,HU,Debrecen,o3,65,2.3,18.9,12.0,193.29
3,12/22/2019,HU,Debrecen,o3,45,11.3,24.1,18.6,110.54
4,1/5/2020,HU,Debrecen,o3,34,1.6,26.4,16.7,586.81
...,...,...,...,...,...,...,...,...,...
578221,11/29/2019,HR,Split,wind-speed,72,0.5,6.9,1.5,15.23
578222,12/7/2019,HR,Split,wind-speed,69,0.2,3.0,1.0,4.68
578223,9/30/2019,HR,Split,wind-speed,72,0.2,3.8,1.8,9.14
578224,12/22/2019,HR,Split,wind-speed,72,2.0,11.5,6.6,65.17


In [44]:
# merge data 
df = df_2020.append(df_2019Q4)
# drop non-US cities
#df.drop(df[df['Country']!='US'].index, inplace = True)
df = df[df['Country'] == 'US']
df.head()

,City,Country,Date,Specie,Unnamed: 0,count,max,median,min,variance
247447,Oklahoma City,US,2020-03-24 00:00:00,no2,247447.0,39,21.3,2.8,1.0,404.46
247448,Oklahoma City,US,2020-03-30 00:00:00,no2,247448.0,44,23.2,7.5,2.8,284.04
247449,Oklahoma City,US,2020-03-31 00:00:00,no2,247449.0,39,20.4,6.5,1.9,270.70
247450,Oklahoma City,US,2020-01-13 00:00:00,no2,247450.0,48,17.6,7.5,2.8,178.59
247451,Oklahoma City,US,2020-01-23 00:00:00,no2,247451.0,47,13.0,3.8,1.0,104.34


In [45]:
# drop irrelevant Species
df['Specie'].unique()
bad_species = ['wind gust', 'temperature', 'wind speed', 'wind-gust', 'pressure', 'dew', 'wind-speed',
       'humidity', 'precipitation', 'wd']
        
df = df[~df.Specie.isin(bad_species)]

In [46]:
# for purpose of using aqi package
d = {}
d['pm25'] = aqi.POLLUTANT_PM25
d['pm10'] = aqi.POLLUTANT_PM10
d['no2'] = aqi.POLLUTANT_NO2_1H
d['o3'] = aqi.POLLUTANT_O3_8H
d['so2'] = aqi.POLLUTANT_SO2_1H
d['co'] = aqi.POLLUTANT_CO_8H
df = df.drop(df[(df['Specie'] == 'co') & (df['median'] == 58.0)].index)

In [47]:
def compute_iaqi(row):
    if(row['Specie'] == 'o3'):
        row['median']/= 1000
    return aqi.to_iaqi(d[row['Specie']], row['median'], algo=aqi.ALGO_EPA)
df['iaqi'] = df.apply(lambda x: compute_iaqi(x),axis=1)

In [48]:
compute_iaqi(df.iloc[0])

Decimal('2')

In [49]:
# take max to get final AQI 
df.sort_values(by=['City','Date'])

df_new = df.drop(['Country','Specie','count','min','max','median','variance'], axis = 1)
df_new = df_new.groupby(['Date','City']).max()

print(type(df_new.axes[1]))
def AQI(row):
    #print(df_new.loc[(row['Date'],row['City'])]["iaqi"])
    return df_new.loc[(row['Date'],row['City'])]["iaqi"]

df['AQI'] = df.apply(AQI,axis=1)

<class 'pandas.core.indexes.base.Index'>


In [50]:
df = df.sort_values(by=['City','Date'])
print(df['City'].unique())

['Albuquerque' 'Atlanta' 'Austin' 'Baltimore' 'Boise' 'Boston' 'Brooklyn'
 'Charlotte' 'Chicago' 'Columbia' 'Columbus' 'Dallas' 'Denver' 'Detroit'
 'El Paso' 'Fort Worth' 'Fresno' 'Hartford' 'Honolulu' 'Houston'
 'Indianapolis' 'Jackson' 'Jacksonville' 'Las Vegas' 'Little Rock'
 'Los Angeles' 'Madison' 'Manhattan' 'Memphis' 'Miami' 'Milwaukee'
 'Nashville' 'Oakland' 'Oklahoma City' 'Omaha' 'Philadelphia' 'Phoenix'
 'Portland' 'Providence' 'Queens' 'Raleigh' 'Richmond' 'Sacramento'
 'Saint Paul' 'Salem' 'Salt Lake City' 'San Antonio' 'San Diego'
 'San Francisco' 'San Jose' 'Seattle' 'Springfield' 'Staten Island'
 'Tallahassee' 'The Bronx' 'Tucson' 'Washington D.C.']


In [51]:
# df.to_csv("US_AQI.csv")

In [52]:
df.head()

,City,Country,Date,Specie,Unnamed: 0,count,max,median,min,variance,iaqi,AQI
294541,Albuquerque,US,2019-12-30 00:00:00,co,294541.0,48,5.8,2.4,1.0,16.15,26,59
294802,Albuquerque,US,2019-12-30 00:00:00,o3,294802.0,63,36.1,24.8,0.8,1044.51,20,59
295143,Albuquerque,US,2019-12-30 00:00:00,pm10,295143.0,72,50.0,9.0,2.0,888.29,8,59
295469,Albuquerque,US,2019-12-30 00:00:00,pm25,295469.0,72,64.0,16.0,6.0,2142.06,59,59
295542,Albuquerque,US,2019-12-30 00:00:00,no2,295542.0,24,22.3,6.5,2.8,422.85,6,59


In [9]:
### City -> County 

d_tocounty = {}
d_tocounty['Albuquerque'] = 'Bernalillo'
d_tocounty['Atlanta'] = 'Fulton'

In [10]:
# COVID data: confirmed 
df_confirmed = pd.read_csv("time_series_covid19_confirmed_US.csv")
df_confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,133,133,133,133,135,135,136,136,136,136
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,11,11,11,11,13,13,13,14,14,14
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,788,897,903,923,974,1043,1068,1118,1213,1252
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,51,51,51,51,51,51,51,53,53,53


In [13]:
df_confirmed = df_confirmed.drop(['UID','iso2','iso3','code3','FIPS'],axis=1)
df_confirmed = df_confirmed.dropna()

In [14]:
df_confirmed.transpose()

,5,6,7,8,9,10,11,12,13,14,...,3250,3251,3253,3254,3255,3256,3257,3258,3259,3260
Admin2,Autauga,Baldwin,Barbour,Bibb,Blount,Bullock,Butler,Calhoun,Chambers,Cherokee,...,Unassigned,Unassigned,Michigan Department of Corrections (MDOC),Federal Correctional Institution (FCI),Bear River,Central Utah,Southeast Utah,Southwest Utah,TriCounty,Weber-Morgan
Province_State,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,...,Wisconsin,Wyoming,Michigan,Michigan,Utah,Utah,Utah,Utah,Utah,Utah
Country_Region,US,US,US,US,US,US,US,US,US,US,...,US,US,US,US,US,US,US,US,US,US
Lat,32.5395,30.7277,31.8683,32.9964,33.9821,32.1003,31.753,33.7748,32.9136,34.1781,...,0,0,0,0,41.5211,39.3723,38.9962,37.8545,40.1249,41.2712
Long_,-86.6441,-87.7221,-85.3871,-87.1251,-86.5679,-85.7127,-86.6806,-85.8263,-85.3907,-85.6064,...,0,0,0,0,-113.083,-111.576,-110.701,-111.442,-109.517,-111.915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/16/20,26,101,14,22,18,8,11,62,231,11,...,0,0,472,36,0,0,0,0,0,0
4/17/20,26,103,15,24,20,8,16,63,236,12,...,0,0,514,44,0,0,0,0,0,0
4/18/20,25,109,18,26,20,9,13,66,240,12,...,0,0,550,45,0,0,0,0,0,0
4/19/20,26,112,20,28,21,9,14,71,245,12,...,0,0,556,51,54,15,6,66,10,119
